In [ ]:
# import packages
import numpy as np
import pandas as pd
import nltk
import string
import re
from collections import Counter
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, decomposition
import xgboost as xgb 


In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Winter Data Comp/dataset/final_dataset.csv')
test = pd.read_csv('/content/drive/MyDrive/Winter Data Comp/dataset/final_dataset_test.csv')

In [ ]:
train.shape

(41157, 16)

## deal with the missing value


In [ ]:
train_copy = train.copy()

In [ ]:
train.CleanTweet.isnull().sum()

44

In [ ]:
train.CleanTweet.fillna('na', inplace = True)

In [ ]:
train.CleanTweet.isnull().sum()

0

## vectorize 

In [ ]:
clean_tweet = train.CleanTweet

In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer()
tfidf = feature_extraction.text.TfidfVectorizer()
LSA = decomposition.TruncatedSVD(n_components=100)

In [ ]:
## let's get counts for the first 10 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(clean_tweet[0:10])
example_tfidf = tfidf.fit_transform(clean_tweet[0:10])
example_tfidf_lsa = LSA.fit_transform(example_tfidf)

In [ ]:
example_train_vectors.shape

(10, 125)

In [ ]:
print(example_train_vectors)

  (0, 65)	1
  (1, 4)	1
  (1, 113)	1
  (1, 67)	2
  (1, 43)	1
  (1, 42)	1
  (1, 83)	2
  (1, 72)	2
  (1, 32)	1
  (1, 26)	1
  (1, 59)	1
  (1, 98)	1
  (1, 38)	1
  (1, 19)	1
  (1, 51)	1
  (1, 100)	1
  (1, 74)	1
  (1, 102)	1
  (1, 2)	1
  (1, 85)	1
  (1, 3)	1
  (1, 111)	1
  (1, 93)	1
  (1, 62)	1
  (1, 76)	1
  :	:
  (8, 27)	1
  (8, 88)	1
  (8, 70)	1
  (8, 114)	1
  (8, 120)	1
  (9, 74)	2
  (9, 102)	1
  (9, 31)	1
  (9, 1)	1
  (9, 115)	1
  (9, 12)	1
  (9, 28)	2
  (9, 87)	1
  (9, 107)	1
  (9, 16)	1
  (9, 121)	1
  (9, 81)	1
  (9, 63)	1
  (9, 104)	1
  (9, 71)	1
  (9, 5)	1
  (9, 86)	1
  (9, 54)	1
  (9, 116)	1
  (9, 44)	1


In [ ]:
train.Sentiment.value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [ ]:
# sent_dict = {
#     'Extremely Negative': -1,
#     'Negative':-0.5,
#     'Neutral':0,
#     'Positive': 0.5,
#     'Extremely Positive':1
# }
# Y = train.Sentiment.map(sent_dict)
# Y.value_counts()

 0.5    11422
-0.5     9917
 0.0     7713
 1.0     6624
-1.0     5481
Name: Sentiment, dtype: int64

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
cat = list(np.array(['Extremely Negative','Negative', 'Neutral', 'Positive','Extremely Positive']).reshape(1,5))
ordenc = OrdinalEncoder(categories=cat)

In [ ]:
sentiment = np.array(train.Sentiment).reshape(-1, 1)
Y = ordenc.fit_transform(sentiment)

In [ ]:
train_vectors = count_vectorizer.fit_transform(clean_tweet)
train_tfidf = tfidf.fit_transform(clean_tweet)
train_tfidf_lsa = LSA.fit_transform(train_tfidf)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
mm = MaxAbsScaler()
train_vectors = mm.fit_transform(train_vectors)
train_tfidf = mm.fit_transform(train_tfidf)
train_tfidf_lsa = mm.fit_transform(train_tfidf_lsa)

In [ ]:
print(train_vectors[:10])

  (0, 15023)	0.3333333333333333
  (1, 1832)	0.3333333333333333
  (1, 1915)	1.0
  (1, 1974)	0.3333333333333333
  (1, 5138)	1.0
  (1, 5853)	0.5
  (1, 6184)	0.5
  (1, 7998)	0.5
  (1, 8352)	0.5
  (1, 8547)	0.3333333333333333
  (1, 9946)	0.5
  (1, 13294)	0.5
  (1, 14121)	0.5
  (1, 15263)	1.0
  (1, 15653)	0.6666666666666666
  (1, 15936)	0.1111111111111111
  (1, 16011)	0.2
  (1, 16825)	0.6666666666666666
  (1, 17205)	1.0
  (1, 18265)	1.0
  (1, 19358)	0.3333333333333333
  (1, 19633)	0.5
  (1, 19850)	0.25
  (1, 21212)	0.3333333333333333
  (1, 21447)	0.5
  :	:
  (8, 21735)	0.1111111111111111
  (8, 22478)	0.3333333333333333
  (8, 22679)	0.5
  (8, 23493)	0.5
  (8, 23676)	0.25
  (9, 422)	0.25
  (9, 2324)	0.3333333333333333
  (9, 4506)	0.25
  (9, 4811)	0.3333333333333333
  (9, 5977)	0.6666666666666666
  (9, 6097)	0.25
  (9, 8805)	0.2
  (9, 10884)	0.25
  (9, 14281)	1.0
  (9, 15578)	0.5
  (9, 15936)	0.2222222222222222
  (9, 16548)	0.3333333333333333
  (9, 17329)	0.5
  (9, 17382)	0.5
  (9, 19850)	0.25


## train models

In [ ]:
scoring = ['precision_macro', 'recall_macro','f1_macro','accuracy']

In [ ]:
def train_model(clf,scoring = scoring):
  scores_vect = model_selection.cross_validate(clf, train_vectors, Y, cv=5, scoring=scoring)
  scores_tfidf = model_selection.cross_validate(clf, train_tfidf, Y, cv=5, scoring=scoring)
  scores_tfidf_lsa = model_selection.cross_validate(clf, train_tfidf_lsa, Y, cv=5, scoring=scoring)
  return scores_vect,scores_tfidf,scores_tfidf_lsa
  

### Ridge classifier

In [1]:

ridge_clf = linear_model.RidgeClassifier()

NameError: ignored

In [ ]:
scores_tfidf_lsa = model_selection.cross_validate(ridge_clf, train_tfidf_lsa, Y, cv=5, scoring=scoring)

In [ ]:
scores_vect = model_selection.cross_validate(ridge_clf, train_vectors, Y, cv=5, scoring=scoring)

In [ ]:
scores_tfidf = model_selection.cross_validate(ridge_clf, train_tfidf, Y, cv=5, scoring=scoring)

In [ ]:
scores_tfidf

In [ ]:
scores_tfidf_lsa

In [ ]:
scores_vect

### logistic regression 

In [ ]:
from sklearn.linear_model import LogisticRegression 

In [ ]:
lr_clf = LogisticRegression(multi_class='multinomial',max_iter = 1000)

In [ ]:
lr_vect_scores,lr_tfdif_scores,lr_lsa_scores = train_model(lr_clf)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
lr_vect_scores,lr_tfdif_scores,lr_lsa_scores

({'fit_time': array([27.77092266, 27.59998798, 27.90366077, 27.49973679, 27.86655068]),
  'score_time': array([0.05338097, 0.0327208 , 0.02703166, 0.05403805, 0.02642512]),
  'test_accuracy': array([0.61722546, 0.62038387, 0.61912283, 0.62337505, 0.62130968]),
  'test_f1_macro': array([0.62535141, 0.62902223, 0.6262526 , 0.63154204, 0.627758  ]),
  'test_precision_macro': array([0.62676256, 0.63168212, 0.63411337, 0.64389097, 0.64318147]),
  'test_recall_macro': array([0.62557267, 0.62749352, 0.6226946 , 0.62433407, 0.62112601])},
 {'fit_time': array([20.91737747, 18.78240252, 20.97984385, 20.2730844 , 21.92850208]),
  'score_time': array([0.0263319 , 0.05192113, 0.03587651, 0.02546477, 0.0255239 ]),
  'test_accuracy': array([0.56899903, 0.5798105 , 0.57089054, 0.5758717 , 0.58073138]),
  'test_f1_macro': array([0.57337794, 0.5848293 , 0.56975961, 0.57545153, 0.57943178]),
  'test_precision_macro': array([0.58974866, 0.60021213, 0.59639189, 0.6057699 , 0.61806101]),
  'test_recall_macr

In [ ]:
### random forrest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf =  RandomForestClassifier()

In [ ]:
rf_vect_scores,rf_tfdif_scores,rf_lsa_scores = train_model(rf_clf)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

In [ ]:
rf_vect_scores,rf_tfdif_scores,rf_lsa_scores

({'fit_time': array([139.84388041, 139.99188495, 140.24195862, 138.95062137,
         139.13812017]),
  'score_time': array([0.76959872, 0.78332424, 0.78850269, 0.77373409, 0.79065943]),
  'test_accuracy': array([0.54931973, 0.55624393, 0.55376017, 0.54525574, 0.56518042]),
  'test_f1_macro': array([0.53592371, 0.54337952, 0.53516914, 0.52416028, 0.53974168]),
  'test_precision_macro': array([0.58403953, 0.60048203, 0.59818998, 0.59888183, 0.62323003]),
  'test_recall_macro': array([0.52886179, 0.53383444, 0.527403  , 0.51668974, 0.5343708 ])},
 {'fit_time': array([126.11371064, 126.49980187, 124.52987742, 125.55987191,
         125.96363878]),
  'score_time': array([0.74171686, 0.74871469, 0.76076221, 0.74607325, 0.76366711]),
  'test_accuracy': array([0.5212585 , 0.53717201, 0.54051756, 0.53310655, 0.53711578]),
  'test_f1_macro': array([0.51666107, 0.53050968, 0.53046225, 0.52482996, 0.52435881]),
  'test_precision_macro': array([0.55063222, 0.56925426, 0.58401877, 0.5826479 , 0.591

### ExtraTree

In [ ]:
from sklearn.tree import ExtraTreeClassifier

In [ ]:
et_clf = ExtraTreeClassifier()

In [ ]:
et_vect_scores,et_tfdif_scores,et_lsa_scores = train_model(et_clf)

In [ ]:
et_vect_scores,et_tfdif_scores,et_lsa_scores

({'fit_time': array([2.44577265, 2.37982368, 2.49295664, 2.33969426, 2.36174607]),
  'score_time': array([0.02966714, 0.03048468, 0.03045559, 0.02828193, 0.03035688]),
  'test_accuracy': array([0.38896987, 0.38799806, 0.40796987, 0.39557769, 0.41428745]),
  'test_f1_macro': array([0.38470892, 0.38166439, 0.39943261, 0.39114269, 0.40424429]),
  'test_precision_macro': array([0.38685546, 0.38435147, 0.40351261, 0.39800214, 0.41085157]),
  'test_recall_macro': array([0.38434859, 0.38178835, 0.40085148, 0.39027886, 0.40531569])},
 {'fit_time': array([2.47183752, 2.45227361, 2.4099369 , 2.43151712, 2.64103603]),
  'score_time': array([0.03030992, 0.03107619, 0.03130937, 0.03015852, 0.03162551]),
  'test_accuracy': array([0.3691691 , 0.38945578, 0.38816669, 0.36945693, 0.38537237]),
  'test_f1_macro': array([0.36520295, 0.38691568, 0.38144131, 0.362564  , 0.38004515]),
  'test_precision_macro': array([0.36685691, 0.38754123, 0.38498029, 0.36638541, 0.38781809]),
  'test_recall_macro': array(

### XGBoost classifier

In [ ]:
xgb_clf = xgb.XGBClassifier(random_state=42)

In [ ]:
xgb_vect_scores,xgb_tfdif_scores,xgb_lsa_scores = train_model(xgb_clf)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

In [ ]:
xgb_vect_scores,xgb_tfdif_scores,xgb_lsa_scores

({'fit_time': array([24.09523964, 24.11087775, 24.03546023, 24.19133329, 24.11337495]),
  'score_time': array([0.15382552, 0.15363479, 0.15531158, 0.15743256, 0.15405583]),
  'test_accuracy': array([0.47789116, 0.49088921, 0.48171547, 0.46956627, 0.49641599]),
  'test_f1_macro': array([0.48036228, 0.49320807, 0.48109248, 0.47268064, 0.49614066]),
  'test_precision_macro': array([0.51154601, 0.53137745, 0.53482155, 0.51987038, 0.55775679]),
  'test_recall_macro': array([0.47174438, 0.48076624, 0.46850718, 0.45719121, 0.48488394])},
 {'fit_time': array([42.22045231, 42.30255866, 42.39415884, 42.45081782, 42.44201946]),
  'score_time': array([0.19125175, 0.19115424, 0.19209385, 0.20026755, 0.18941569]),
  'test_accuracy': array([0.46853741, 0.48493683, 0.47758474, 0.45875349, 0.48900498]),
  'test_f1_macro': array([0.47153447, 0.48750622, 0.47463918, 0.46235589, 0.48818031]),
  'test_precision_macro': array([0.50616708, 0.52419925, 0.52751371, 0.50848455, 0.55233983]),
  'test_recall_macr